In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
  
# Read the CSV file.
data = pd.read_csv("CTG.csv", skiprows=1)

# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=69)


# look at the data 
X.head()

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
#lets create a dummy classifier
clf = DummyClassifier(strategy='most_frequent')
# test the classifier
cross_val_score(clf, Xtrain, Ytrain)

In [ ]:
#import a bunch of other classifiers
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
MAX_ITER = 0x7fffffff
# create a dictionary of classifiers
clfs = {
    'Dummy': DummyClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Perceptron': Perceptron(max_iter=MAX_ITER),
    'Logistic Regression': LogisticRegression(max_iter=MAX_ITER),
    'Linear SVM': LinearSVC(max_iter=MAX_ITER),
    'Neural Network': MLPClassifier(max_iter=MAX_ITER)
}

# loop through the classifiers and test them
for name, clf in clfs.items():
    score= cross_val_score(clf, Xtrain, Ytrain)
    print("----------------------------------")
    print(f'{name}: {score.mean()}')
    print("----------------------------------")

